In [1]:

import tdmclient.notebook
await tdmclient.notebook.start()
from test import *


In [2]:
printing()

coucou


In [ ]:
%%run_python --wait

@onevent 
def button_right():
    global motor_left_target, motor_right_target
    print('salut')
    motor_left_target = 100
    motor_right_target = -100